# Análise preditiva

Siga estes passos:
- Ajuste `TARGET_COLUMN` na próxima célula (ou deixe `None` para autodetecção).
- Execute as células em ordem, de cima para baixo.
- Os arquivos `.xlsx` serão lidos do diretório atual e da pasta `Dados/`.
- Saídas: modelos em `models/` e métricas em `artifacts/`.


In [1]:
# Ajuda rápida: visualizar colunas e candidatos a alvo
print('Dica: execute primeiro as células de configuração e de carregamento de dados. Depois use a célula de "Preview de colunas" mais abaixo.')


Dica: execute primeiro as células de configuração e de carregamento de dados. Depois use a célula de "Preview de colunas" mais abaixo.


In [2]:
# Configuração inicial
# Ajuste estes parâmetros conforme seu caso
TARGET_COLUMN = 'INVESTIMENTO TOTAL (BI)'
DATE_COLUMNS_CANDIDATES = ['data', 'dt', 'date', 'competencia', 'mes', 'mês', 'ano']
ID_COLUMNS_CANDIDATES = ['id', 'codigo', 'código', 'cod', 'cód', 'cnpj', 'cpf']
TASK_TYPE = 'regression'  # regressão sobre INVESTIMENTO TOTAL (BI)
SHEET_NAME_PREFERRED = 'todas as tabelas'
PREFERRED_COLUMNS = [
	'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 'EXTENSÃO (km)', '(km)%', '(km)% EXEC', '(km)% PLAN',
	'Ext. (km)', 'FINANCEIRO (R$)', 'FINANCEIRO PLAN (R$)', 'PERCENTUAL (%)', 'PERCENTUAL (%) EXEC', 'PERCENTUAL (%) PLAN'
]
PRIORITY_TARGETS = ['INVESTIMENTO TOTAL (BI)', 'PERCENTUAL (%)', 'FINANCEIRO (R$)']
TEST_SIZE = 0.1
VAL_SIZE = 0.1
RANDOM_STATE = 42
MAX_EPOCHS = 200
BATCH_SIZE = 64
PATIENCE = 20
MODEL_DIR = 'models'
ARTIFACTS_DIR = 'artifacts'
# Limite opcional de linhas para acelerar (None desativa)
MAX_ROWS = None


# Imports
import os
import sys
import math
import json
import glob
import shutil
from pathlib import Path

import numpy as np
import pandas as pd

# Optional installs (executa apenas se faltar pacote)
try:
	import sklearn  # type: ignore
except Exception:
	!pip -q install scikit-learn
	import sklearn  # type: ignore

try:
	import tensorflow as tf  # type: ignore
except Exception:
	!pip -q install tensorflow==2.*
	import tensorflow as tf  # type: ignore

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, f1_score, classification_report

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

print('Versões:')
print('Python:', sys.version)
print('Pandas:', pd.__version__)
print('Scikit-learn:', sklearn.__version__)
print('TensorFlow:', tf.__version__)


Versões:
Python: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
Pandas: 2.3.2
Scikit-learn: 1.7.2
TensorFlow: 2.20.0


In [3]:
# Preferência por alvos informados e seleção automática
if 'df_raw' in globals():
	if TARGET_COLUMN is None:
		for t in PRIORITY_TARGETS:
			if t in df_raw.columns:
				TARGET_COLUMN = t
				print('TARGET_COLUMN escolhido por prioridade =', TARGET_COLUMN)
				break
		if TARGET_COLUMN is None:
			print('Nenhum alvo prioritário encontrado; manter heurística padrão.')
else:
	print('Carregue os dados primeiro.')


Carregue os dados primeiro.


In [4]:
# Discover and load Excel files
from datetime import datetime

ROOT = Path('.')
DATA_DIRS = [ROOT, ROOT / 'Dados']
excel_files = []
for d in DATA_DIRS:
	if d.exists():
		excel_files.extend(sorted(map(Path, glob.glob(str(d / '*.xlsx')))))

if not excel_files:
	raise FileNotFoundError('No .xlsx files found in project root or Dados/.')

print(f'Found {len(excel_files)} Excel files:')
for p in excel_files:
	print('-', p)

# Load preferred sheet if present, otherwise all sheets
frames = []
for fp in excel_files:
	try:
		xl = pd.ExcelFile(fp)
		sheets = xl.sheet_names
		if SHEET_NAME_PREFERRED in sheets:
			candidate_sheets = [SHEET_NAME_PREFERRED]
		else:
			candidate_sheets = sheets
		for sheet in candidate_sheets:
			try:
				df = xl.parse(sheet)
				df['__source_file'] = fp.name
				df['__source_sheet'] = sheet
				frames.append(df)
			except Exception as e:
				print(f'Warning parsing sheet {sheet} in {fp.name}: {e}')
	except Exception as e:
		print(f'Warning opening {fp.name}: {e}')

if not frames:
	raise RuntimeError('Could not read any sheet from Excel files.')

df_raw = pd.concat(frames, ignore_index=True)
print('Raw shape:', df_raw.shape)
print('Columns:', list(df_raw.columns)[:30])

df_raw.head(3)


Found 7 Excel files:
- Planilha Monitoramento_PPI - Carregamento - BI 2025 - 6.xlsx
- Planilha Monitoramento_PPI - Carregamento - BI 2025 - 7.xlsx
- Planilha Monitoramento_PPI - Carregamento - BI 2025 - 8.xlsx
- Dados\Planilha Monitoramento_PPI - Carregamento - BI 2025 - 1.xlsx
- Dados\Planilha Monitoramento_PPI - Carregamento - BI 2025 - 2.xlsx
- Dados\Planilha Monitoramento_PPI - Carregamento - BI 2025 - 3.xlsx
- Dados\Planilha Monitoramento_PPI - Carregamento - BI 2025 - 6.xlsx
Raw shape: (606526, 49)
Columns: ['Região', 'BR', 'EMPREENDIMENTO', 'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 'ETAPA', 'SITUAÇÃO', 'EXTENSÃO (km)', 'ESTADO/LOTE', 'PLANILHA', '__source_file', '__source_sheet', 'ID-ÚNICO', 'SETOR', 'UF', 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor', 'ID-ÚNICO2', 'SETOR2', 'EMPREENDIMENTO2']


,Região,BR,EMPREENDIMENTO,PROPONENTE,EXECUTOR (Grupo Controlador),ESTRUTURADOR DO PROJETO,ANO LEILÃO,DATA DE INÍCIO,ANO DA CONCESSÃO,PRAZO (anos),...,(km)% PLAN,Descrição,Ext. (km),FINANCEIRO (R$),FINANCEIRO PLAN (R$),PERCENTUAL (%),PERCENTUAL (%) EXEC,PERCENTUAL (%) PLAN,km (f),km (i)
0,SUL,1.0,VIA SUL,ANTT,CCR,EPL,2018.0,2019-02-15,7.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SD/CO,1.0,CERRADO,ANTT,ECORODOVIAS,NaN,2019.0,2020-01-20,6.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SUL,1.0,VIA COSTEIRA,ANTT,CCR,NaN,2020.0,2020-07-08,5.0,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Preview de colunas (execute após carregar df_raw)
if 'df_raw' not in globals():
	print('Carregue os dados primeiro.')
else:
	print('Colunas (primeiras 30):', list(df_raw.columns)[:30])
	print('Tipos:')
	print(df_raw.dtypes.head(30))
	num_cols = [c for c in df_raw.columns if pd.api.types.is_numeric_dtype(df_raw[c])]
	cat_cols = [c for c in df_raw.columns if c not in num_cols]
	print('Possíveis alvos numéricos:', num_cols[:10])
	print('Possíveis alvos categóricos:', cat_cols[:10])


Colunas (primeiras 30): ['Região', 'BR', 'EMPREENDIMENTO', 'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'INVESTIMENTO TOTAL (BI)', 'ETAPA', 'SITUAÇÃO', 'EXTENSÃO (km)', 'ESTADO/LOTE', 'PLANILHA', '__source_file', '__source_sheet', 'ID-ÚNICO', 'SETOR', 'UF', 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor', 'ID-ÚNICO2', 'SETOR2', 'EMPREENDIMENTO2']
Tipos:
Região                                  object
BR                                     float64
EMPREENDIMENTO                          object
PROPONENTE                              object
EXECUTOR (Grupo Controlador)            object
ESTRUTURADOR DO PROJETO                 object
ANO LEILÃO                             float64
DATA DE INÍCIO                  datetime64[ns]
ANO DA CONCESSÃO                       float64
PRAZO (anos)                           float64
CAPEX (BI)                             float64
OP

In [6]:
# Correção de formatos BR para colunas preferidas
# Remove R$, pontos de milhar, troca vírgula por ponto e trata %

def br_to_float(series: pd.Series) -> pd.Series:
	if pd.api.types.is_numeric_dtype(series):
		return series
	s = series.astype(str).str.strip()
	s = s.str.replace(r'\s', '', regex=True)
	s = s.str.replace('R$', '', regex=False)
	s = s.str.replace('.', '', regex=False)
	s = s.str.replace(',', '.', regex=False)
	# Percentuais: transformar "10%" em 0.10
	is_percent = s.str.contains('%', regex=False)
	s = s.str.replace('%', '', regex=False)
	out = pd.to_numeric(s, errors='coerce')
	out = out.where(~is_percent, out / 100.0)
	return out

if 'df_raw' in globals():
	for col in PREFERRED_COLUMNS:
		if col in df_raw.columns:
			try:
				df_raw[col] = br_to_float(df_raw[col])
			except Exception as e:
				print(f'Falha ao converter {col}: {e}')
	print('Correção BR aplicada onde possível nas colunas preferidas.')
else:
	print('Carregue os dados primeiro.')


Correção BR aplicada onde possível nas colunas preferidas.


In [7]:
# Infer target, date and id columns helpers

def normalize_name(name: str) -> str:
	return str(name).strip().lower().replace('\n',' ').replace('\r',' ')

norm_cols = {c: normalize_name(c) for c in df_raw.columns}

# Try to infer target if not provided
if TARGET_COLUMN is None:
	# Heuristic: last numeric column
	numeric_cols = [c for c in df_raw.columns if pd.api.types.is_numeric_dtype(df_raw[c])]
	TARGET_COLUMN = numeric_cols[-1] if numeric_cols else df_raw.columns[-1]
	print('Auto-selected TARGET_COLUMN =', TARGET_COLUMN)
else:
	assert TARGET_COLUMN in df_raw.columns, f'TARGET_COLUMN {TARGET_COLUMN} not in data.'

# Detect date-like columns
candidate_date_cols = []
for c, n in norm_cols.items():
	if any(tok in n for tok in DATE_COLUMNS_CANDIDATES):
		candidate_date_cols.append(c)

# Also include truly datetime types
for c in df_raw.columns:
	if pd.api.types.is_datetime64_any_dtype(df_raw[c]):
		candidate_date_cols.append(c)

candidate_date_cols = list(dict.fromkeys(candidate_date_cols))  # unique
print('Candidate date columns:', candidate_date_cols)

# Detect id-like columns
candidate_id_cols = []
for c, n in norm_cols.items():
	if any(tok in n for tok in ID_COLUMNS_CANDIDATES):
		candidate_id_cols.append(c)
print('Candidate id columns:', candidate_id_cols)


Candidate date columns: ['ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)']
Candidate id columns: ['ID-ÚNICO', 'ID-ÚNICO2', 'ID-ÚNICO-TT']


In [8]:
# Build feature table
df = df_raw.copy()

# Clean target FIRST: coerce to numeric and drop NaN/inf
if TARGET_COLUMN in df.columns:
	df[TARGET_COLUMN] = pd.to_numeric(df[TARGET_COLUMN], errors='coerce')
	mask_target = df[TARGET_COLUMN].apply(lambda v: pd.notna(v) and np.isfinite(v))
	df = df[mask_target]

# Stop early if no rows remain
if len(df) == 0:
	raise SystemExit('Sem linhas com alvo válido após limpeza do alvo.')

# Parse first date column if available
main_date_col = None
for c in candidate_date_cols:
	try:
		df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=True, infer_datetime_format=True)
		if df[c].notna().sum() > 0 and main_date_col is None:
			main_date_col = c
	except Exception:
		pass

# Optional row limit to speed up (apply after knowing main_date_col)
if 'MAX_ROWS' in globals() and MAX_ROWS is not None:
	if main_date_col and main_date_col in df.columns:
		df = df.sort_values(by=main_date_col).tail(int(MAX_ROWS))
	else:
		df = df.sample(n=int(MAX_ROWS), random_state=RANDOM_STATE) if len(df) > MAX_ROWS else df

if main_date_col:
	df['__year'] = df[main_date_col].dt.year
	df['__month'] = df[main_date_col].dt.month
	df['__day'] = df[main_date_col].dt.day
	df['__quarter'] = df[main_date_col].dt.quarter
	print('Using time column:', main_date_col)
else:
	print('No usable time column found; will use random split.')

# Drop purely identifier columns and technical columns
cols_to_drop = set(candidate_id_cols + ['__source_file', '__source_sheet'])
feature_cols = [c for c in df.columns if c not in cols_to_drop and c != TARGET_COLUMN]

# Separate X, y
X = df[feature_cols]
y = df[TARGET_COLUMN]

print('Feature columns (first 30):', feature_cols[:30])
print('X shape:', X.shape, 'y shape:', y.shape)

# Determine task type
if TASK_TYPE is None:
	if pd.api.types.is_numeric_dtype(y):
		unique_vals = pd.Series(y).nunique(dropna=True)
		TASK_TYPE = 'regression' if unique_vals > 15 else 'classification'
	else:
		TASK_TYPE = 'classification'
print('TASK_TYPE =', TASK_TYPE)


Using time column: ANO LEILÃO
Feature columns (first 30): ['Região', 'BR', 'EMPREENDIMENTO', 'PROPONENTE', 'EXECUTOR (Grupo Controlador)', 'ESTRUTURADOR DO PROJETO', 'ANO LEILÃO', 'DATA DE INÍCIO', 'ANO DA CONCESSÃO', 'PRAZO (anos)', 'CAPEX (BI)', 'OPEX (BI)', 'ETAPA', 'SITUAÇÃO', 'EXTENSÃO (km)', 'ESTADO/LOTE', 'PLANILHA', 'SETOR', 'UF', 'Atributo.1', 'Atributo.2', 'Atributo.3', 'Valor', 'SETOR2', 'EMPREENDIMENTO2', 'Descrição2', 'Ext.(km)2', 'FINANCEIRO PLAN (R$)2', 'FINANCEIRO(R$)2', '(km)% PLAN2']
X shape: (406, 47) y shape: (406,)
TASK_TYPE = regression


C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\3808863425.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=True, infer_datetime_format=True)
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\3808863425.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', dayfirst=True, infer_datetime_format=True)
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\3808863425.py:18: UserWarning: The argument 'infer_datetime_format' is deprecated a

In [9]:
# Preprocessing with ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

numeric_features = [c for c in X.columns if pd.api.types.is_numeric_dtype(X[c])]
categorical_features = [c for c in X.columns if c not in numeric_features]

# Ensure categorical columns are uniformly strings (preserve NaN)
def to_str_preserve_nan(df_in):
	return df_in.applymap(lambda v: str(v) if pd.notna(v) else v)

numeric_pipeline = Pipeline(steps=[
	('imputer_median', SimpleImputer(strategy='median')),
	('imputer_zero', SimpleImputer(strategy='constant', fill_value=0)),
	('scaler', StandardScaler(with_mean=True, with_std=True)),
])

categorical_pipeline = Pipeline(steps=[
	('to_str', FunctionTransformer(to_str_preserve_nan)),
	('imputer', SimpleImputer(strategy='most_frequent')),
	('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, min_frequency=0.01)),
])

preprocessor = ColumnTransformer(
	transformers=[
		('num', numeric_pipeline, numeric_features),
		('cat', categorical_pipeline, categorical_features),
	],
	remainder='drop',
)

print('Numeric features:', len(numeric_features), 'Categorical features:', len(categorical_features))


Numeric features: 22 Categorical features: 25


In [10]:
# Train/Validation/Test split (time-aware if possible)
from sklearn.model_selection import train_test_split

if main_date_col:
	# Sort by time and split by index ratios
	df_sorted = df.sort_values(by=main_date_col)
	X_sorted = df_sorted[feature_cols]
	y_sorted = df_sorted[TARGET_COLUMN]
	
	n_total = len(df_sorted)
	n_test = max(1, int(math.floor(TEST_SIZE * n_total))) if n_total >= 3 else 1
	n_val_pool = max(0, n_total - n_test)
	n_val = max(1, int(math.floor(VAL_SIZE * n_val_pool))) if n_val_pool >= 3 else (1 if n_val_pool >= 2 else 0)
	train_end = n_total - n_test - n_val
	if train_end <= 0:
		# Fallback to random split to avoid empty sets
		X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=min(0.2, max(0.1, TEST_SIZE)), random_state=RANDOM_STATE, stratify=(y if TASK_TYPE=='classification' else None))
		X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=min(0.2, max(0.1, VAL_SIZE)), random_state=RANDOM_STATE, stratify=(y_temp if TASK_TYPE=='classification' else None))
	else:
		X_train = X_sorted.iloc[: train_end]
		y_train = y_sorted.iloc[: train_end]
		X_val = X_sorted.iloc[train_end : train_end + n_val]
		y_val = y_sorted.iloc[train_end : train_end + n_val]
		X_test = X_sorted.iloc[n_total - n_test :]
		y_test = y_sorted.iloc[n_total - n_test :]
else:
	X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=(y if TASK_TYPE=='classification' else None))
	X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=(y_temp if TASK_TYPE=='classification' else None))

print('Train:', X_train.shape, 'Val:', X_val.shape, 'Test:', X_test.shape)

# Guard against empty splits
assert len(X_train) > 0 and len(X_val) > 0 and len(X_test) > 0, 'One of the splits is empty after splitting.'

# Fit preprocessor on train only
X_train_t = preprocessor.fit_transform(X_train)
X_val_t = preprocessor.transform(X_val)
X_test_t = preprocessor.transform(X_test)

input_dim = X_train_t.shape[1]
print('Transformed dims:', X_train_t.shape, X_val_t.shape, X_test_t.shape)


Train: (330, 47) Val: (36, 47) Test: (40, 47)
Transformed dims: (330, 142) (36, 142) (40, 142)


C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Ext.(km)2' 'FINANCEIRO PLAN (R$)2' 'FINANCEIRO(R$)2' '(km)% PLAN2'
 '(km)% EXEC2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\2130103296.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_in.applymap(lambda v: str(v) if pd.notna(v) else v)
C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['SETOR' 'UF' 'Atributo.1' 'Atributo.2' 'Atributo.3' 'Valor' 'SETOR2'
 'EMPREENDIMENTO2' 'Descrição2' 'Descrição']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(
C:\Users\carlos.ramos\AppData\Local\Pr

In [11]:
# Define and train Keras model
from tensorflow import keras
from tensorflow.keras import layers


def build_mlp(input_dim: int, task: str) -> keras.Model:
	units = max(16, min(512, int(2 * math.sqrt(input_dim + 1)) * 16))
	inputs = keras.Input(shape=(input_dim,))
	x = layers.Dense(units, activation='relu')(inputs)
	x = layers.BatchNormalization()(x)
	x = layers.Dropout(0.2)(x)
	x = layers.Dense(units // 2, activation='relu')(x)
	x = layers.BatchNormalization()(x)
	x = layers.Dropout(0.2)(x)
	if task == 'regression':
		outputs = layers.Dense(1, activation='linear')(x)
		loss = 'mae'
		metrics = ['mae', 'mse']
	else:
		# detect classes
		if pd.api.types.is_numeric_dtype(y_train):
			classes = np.unique(y_train.dropna())
			num_classes = len(classes)
		else:
			classes = np.unique(pd.Series(y_train).astype(str))
			num_classes = len(classes)
		outputs = layers.Dense(num_classes, activation='softmax')(x)
		loss = 'sparse_categorical_crossentropy'
		metrics = ['accuracy']
	model = keras.Model(inputs, outputs)
	model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss=loss, metrics=metrics)
	return model

model = build_mlp(input_dim, TASK_TYPE)

callbacks = [
	keras.callbacks.EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
]

# Prepare targets
if TASK_TYPE == 'classification':
	# Map labels to integers for sparse_categorical_crossentropy
	label_series = pd.Series(y_train)
	label_to_index = {label: idx for idx, label in enumerate(sorted(label_series.dropna().unique(), key=lambda x: str(x)))}
	index_to_label = {v: k for k, v in label_to_index.items()}

y_train_arr = y_train.map(label_to_index).values if TASK_TYPE=='classification' else y_train.values

y_val_arr = y_val.map(label_to_index).values if TASK_TYPE=='classification' else y_val.values

y_test_arr = y_test.map(label_to_index).values if TASK_TYPE=='classification' else y_test.values

history = model.fit(
	X_train_t, y_train_arr,
	validation_data=(X_val_t, y_val_arr),
	epochs=MAX_EPOCHS,
	batch_size=BATCH_SIZE,
	verbose=1,
	callbacks=callbacks,
)

print('Best val metrics:', min(history.history['val_loss']))


Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 10.6064 - mae: 10.6064 - mse: 167.4851 - val_loss: 10.0564 - val_mae: 10.0564 - val_mse: 109.7840
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 10.5747 - mae: 10.5747 - mse: 154.3349 - val_loss: 9.8936 - val_mae: 9.8936 - val_mse: 106.3346
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 10.4518 - mae: 10.4518 - mse: 145.2920 - val_loss: 9.7349 - val_mae: 9.7349 - val_mse: 103.1143
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 10.2575 - mae: 10.2575 - mse: 141.0740 - val_loss: 9.5872 - val_mae: 9.5872 - val_mse: 100.2622
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 10.1769 - mae: 10.1769 - mse: 140.8357 - val_loss: 9.4567 - val_mae: 9.4567 - val_mse: 97.8438
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 10.0800 - mae: 10.0800 - mse: 138.8661 - val_loss: 9.3062 - val_mae: 9.3062 - val_mse: 95.0346
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 9.9468 - mae: 9.94

In [12]:
# Evaluate and save artifacts
import time
import joblib

# Predictions with NaN-safe evaluation
if TASK_TYPE == 'regression':
	y_pred = model.predict(X_test_t, verbose=0).reshape(-1)
	y_true_np = y_test.to_numpy()
	mask = (~np.isnan(y_true_np)) & (~np.isnan(y_pred))
	if mask.sum() == 0:
		raise ValueError('No valid test rows to evaluate after NaN filtering.')
	mae_val = float(mean_absolute_error(y_true_np[mask], y_pred[mask]))
	r2_val = float(r2_score(y_true_np[mask], y_pred[mask]))
	metrics_out = {'mae': mae_val, 'r2': r2_val}
else:
	proba = model.predict(X_test_t, verbose=0)
	y_pred_idx = proba.argmax(axis=1)
	mask = ~pd.isna(y_test)
	if mask.sum() == 0:
		raise ValueError('No valid test rows to evaluate after NaN filtering (classification).')
	y_true = y_test[mask]
	y_pred = pd.Series(y_pred_idx)[mask.to_numpy()].map(index_to_label)
	metrics_out = {
		'accuracy': float(accuracy_score(y_true, y_pred)),
		'f1_macro': float(f1_score(y_true, y_pred, average='macro')),
	}
	print(classification_report(y_true, y_pred))

print('Test metrics:', metrics_out)

# Save artifacts
run_id = time.strftime('%Y%m%d-%H%M%S')
model_path = Path(MODEL_DIR) / f'model_{TASK_TYPE}_{run_id}.keras'
preproc_path = Path(MODEL_DIR) / f'preprocessor_{run_id}.joblib'
report_path = Path(ARTIFACTS_DIR) / f'metrics_{TASK_TYPE}_{run_id}.json'
labelmap_path = Path(MODEL_DIR) / f'labelmap_{run_id}.json'

model.save(model_path)
joblib.dump(preprocessor, preproc_path)

with open(report_path, 'w', encoding='utf-8') as f:
	json.dump(metrics_out, f, ensure_ascii=False, indent=2)

if TASK_TYPE == 'classification':
	with open(labelmap_path, 'w', encoding='utf-8') as f:
		json.dump({'label_to_index': {str(k): int(v) for k, v in label_to_index.items()}, 'index_to_label': {str(k): str(v) for k, v in index_to_label.items()}}, f, ensure_ascii=False, indent=2)

print('Saved:')
print('-', model_path)
print('-', preproc_path)
print('-', report_path)
if TASK_TYPE == 'classification':
	print('-', labelmap_path)


Test metrics: {'mae': 1.0935167293548589, 'r2': 0.7676028264513364}
Saved:
- models\model_regression_20250923-110842.keras
- models\preprocessor_20250923-110842.joblib
- artifacts\metrics_regression_20250923-110842.json


In [13]:
# Example: inference on new data row
# Provide a single-row DataFrame with same columns as X
example = X.head(1).copy()
X_new_t = preprocessor.transform(example)
proba_or_pred = model.predict(X_new_t, verbose=0)
print('Prediction output shape:', proba_or_pred.shape)


Prediction output shape: (1, 1)


C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Ext.(km)2' 'FINANCEIRO PLAN (R$)2' 'FINANCEIRO(R$)2' '(km)% PLAN2'
 '(km)% EXEC2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\2130103296.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_in.applymap(lambda v: str(v) if pd.notna(v) else v)
C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['SETOR' 'UF' 'Atributo.1' 'Atributo.2' 'Atributo.3' 'Valor' 'SETOR2'
 'EMPREENDIMENTO2' 'Descrição2' 'Descrição']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


In [14]:
# Inference: carregar dados novos e salvar predicoes.xlsx
import argparse
import pandas as pd
import joblib
from tensorflow import keras
from pathlib import Path

# Parâmetros
INPUT_PATH = None  # ex.: 'novos_dados.xlsx' ou 'novos_dados.csv'; se None, usa X.head(100)
OUTPUT_PATH = 'predicoes.xlsx'

# Localizar últimos artifacts gerados
models_dir = Path(MODEL_DIR)
artifacts_dir = Path(ARTIFACTS_DIR)
latest_model = sorted(models_dir.glob('model_regression_*.keras') if TASK_TYPE=='regression' else models_dir.glob('model_classification_*.keras'))[-1]
latest_preproc = sorted(models_dir.glob('preprocessor_*.joblib'))[-1]

print('Usando modelo:', latest_model)
print('Usando preprocessor:', latest_preproc)

preprocessor = joblib.load(latest_preproc)
model = keras.models.load_model(latest_model)

# Carregar dados novos
if INPUT_PATH is None:
	print('INPUT_PATH não definido; usando amostra de X.head(100).')
	df_new = X.head(100).copy()
else:
	p = Path(INPUT_PATH)
	if p.suffix.lower() in ['.xlsx', '.xls']:
		df_new = pd.read_excel(p)
	elif p.suffix.lower() == '.csv':
		df_new = pd.read_csv(p)
	else:
		raise ValueError('Formato não suportado: use .xlsx, .xls ou .csv')

# Garantir mesmas colunas de treino (faltantes serão criadas vazias; extras serão ignoradas pelo preprocessor)
for c in [col for col in X.columns if col not in df_new.columns]:
	df_new[c] = pd.NA

X_new_t = preprocessor.transform(df_new[X.columns])
proba_or_pred = model.predict(X_new_t, verbose=0)

if TASK_TYPE == 'regression':
	y_pred = proba_or_pred.reshape(-1)
	df_out = df_new.copy()
	df_out['pred_'+str(TARGET_COLUMN).replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_')] = y_pred
else:
	pred_idx = proba_or_pred.argmax(axis=1)
	# Recarregar labelmap mais recente, se existir
	labelmaps = sorted(models_dir.glob('labelmap_*.json'))
	if labelmaps:
		import json
		with open(labelmaps[-1], 'r', encoding='utf-8') as f:
			m = json.load(f)
			index_to_label = {int(k): v for k, v in m.get('index_to_label', {}).items()}
		labels = [index_to_label.get(int(i), str(i)) for i in pred_idx]
	else:
		labels = pred_idx
	df_out = df_new.copy()
	df_out['pred_label'] = labels

# Salvar (tolerante a arquivo aberto no Windows)
Path(ARTIFACTS_DIR).mkdir(exist_ok=True, parents=True)
output_path = Path(ARTIFACTS_DIR) / OUTPUT_PATH
try:
	if output_path.suffix.lower() == '.csv':
		df_out.to_csv(output_path, index=False)
	else:
		df_out.to_excel(output_path, index=False, engine='openpyxl')
except PermissionError:
	import time
	alt_name = output_path.with_stem(output_path.stem + '_' + time.strftime('%Y%m%d-%H%M%S'))
	if alt_name.suffix.lower() == '.csv':
		df_out.to_csv(alt_name, index=False)
	else:
		df_out.to_excel(alt_name, index=False, engine='openpyxl')
	output_path = alt_name

print('Predições salvas em:', output_path)



Usando modelo: models\model_regression_20250923-110842.keras
Usando preprocessor: models\preprocessor_20250923-110842.joblib
INPUT_PATH não definido; usando amostra de X.head(100).


C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Ext.(km)2' 'FINANCEIRO PLAN (R$)2' 'FINANCEIRO(R$)2' '(km)% PLAN2'
 '(km)% EXEC2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\2130103296.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_in.applymap(lambda v: str(v) if pd.notna(v) else v)
C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['SETOR' 'UF' 'Atributo.1' 'Atributo.2' 'Atributo.3' 'Valor' 'SETOR2'
 'EMPREENDIMENTO2' 'Descrição2' 'Descrição']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Predições salvas em: artifacts\predicoes.xlsx


In [15]:
# Predições futuras para anos específicos (2026–2030) por tabela/ID
import pandas as pd
from pathlib import Path

YEARS_TO_FORECAST = [2026, 2027, 2028, 2029, 2030]
OUTPUT_FUTURE_FILE = 'predicoes_futuras_2026-2030.xlsx'

assert 'df' in globals() and 'X' in globals(), 'Execute as células de carregamento e de engenharia antes.'
assert 'preprocessor' in globals() and 'model' in globals(), 'Treine o modelo antes desta célula.'

# Escolher melhor coluna de identificação disponível
id_candidates = ['ID-ÚNICO-TT', 'ID-ÚNICO', 'ID-ÚNICO2', 'EMPREENDIMENTO', 'ESTADO/LOTE', 'PLANILHA']
id_col = None
for c in id_candidates:
	if c in df.columns:
		id_col = c
		break

# Base: última linha por ID (ou última do conjunto) para usar como "perfil" do projeto/tabela
if id_col is not None:
	base_df = df.dropna(subset=[id_col]).copy()
	if 'main_date_col' in globals() and main_date_col and main_date_col in base_df.columns:
		base_df = base_df.sort_values(by=main_date_col)
		base_df = base_df.groupby(id_col, as_index=False).tail(1)
	else:
		base_df = base_df.drop_duplicates(subset=[id_col], keep='last')
else:
	base_df = df.copy()
	if 'main_date_col' in globals() and main_date_col and main_date_col in base_df.columns:
		base_df = base_df.sort_values(by=main_date_col).tail(1)

if len(base_df) == 0:
	raise SystemExit('Não há linhas base para projetar futuras predições.')

# Criar cenários para os anos desejados, ajustando campos de data/ano relevantes
scenario_frames = []
for year in YEARS_TO_FORECAST:
	f = base_df.copy()
	# Ajustes em colunas temporais comuns
	if 'ANO LEILÃO' in f.columns:
		f['ANO LEILÃO'] = int(year)
	if 'ANO DA CONCESSÃO' in f.columns:
		f['ANO DA CONCESSÃO'] = int(year)
	if 'DATA DE INÍCIO' in f.columns:
		try:
			f['DATA DE INÍCIO'] = pd.Timestamp(int(year), 1, 1)
		except Exception:
			f['DATA DE INÍCIO'] = pd.NaT
	# Se usamos uma coluna temporal principal, sincronizar features derivadas
	if 'main_date_col' in globals() and main_date_col and main_date_col in f.columns:
		try:
			f[main_date_col] = pd.to_datetime(pd.Series([pd.Timestamp(int(year), 1, 1)] * len(f)))
		except Exception:
			pass
	# Rótulo do ano previsto
	f['ANO_PREVISTO'] = int(year)
	scenario_frames.append(f)

df_future = pd.concat(scenario_frames, ignore_index=True)

# Garantir que teremos exatamente as mesmas colunas de entrada do treino
for col in [c for c in X.columns if c not in df_future.columns]:
	df_future[col] = pd.NA

# Transformar e prever
X_future = df_future[X.columns]
X_future_t = preprocessor.transform(X_future)
proba_or_pred = model.predict(X_future_t, verbose=0)

if TASK_TYPE == 'regression':
	y_pred = proba_or_pred.reshape(-1)
	pred_col_name = 'pred_' + str(TARGET_COLUMN).replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_')
	df_future_out = df_future.copy()
	df_future_out[pred_col_name] = y_pred
else:
	pred_idx = proba_or_pred.argmax(axis=1)
	# Mapear rótulos se disponível
	labels = pred_idx
	try:
		from pathlib import Path as _P
		labelmaps = sorted((_P(MODEL_DIR)).glob('labelmap_*.json'))
		if labelmaps:
			import json
			with open(labelmaps[-1], 'r', encoding='utf-8') as f:
				m = json.load(f)
				index_to_label = {int(k): v for k, v in m.get('index_to_label', {}).items()}
			labels = [index_to_label.get(int(i), str(i)) for i in pred_idx]
	except Exception:
		pass
	df_future_out = df_future.copy()
	df_future_out['pred_label'] = labels

# Selecionar colunas chave para saída, mantendo identificadores úteis
id_out_cols = [c for c in ['ANO_PREVISTO', 'ID-ÚNICO-TT', 'ID-ÚNICO', 'EMPREENDIMENTO', 'ESTADO/LOTE', 'UF', 'SETOR', 'PLANILHA'] if c in df_future_out.columns]
value_cols = [c for c in df_future_out.columns if c.startswith('pred_') or c == 'pred_label']
other_cols = []

df_export = df_future_out[id_out_cols + value_cols + other_cols]

# Salvar
Path(ARTIFACTS_DIR).mkdir(exist_ok=True, parents=True)
output_path = Path(ARTIFACTS_DIR) / OUTPUT_FUTURE_FILE
if output_path.suffix.lower() == '.csv':
	df_export.to_csv(output_path, index=False)
else:
	df_export.to_excel(output_path, index=False)

print('Predições futuras salvas em:', output_path)
print('Linhas geradas:', len(df_export))


C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Ext.(km)2' 'FINANCEIRO PLAN (R$)2' 'FINANCEIRO(R$)2' '(km)% PLAN2'
 '(km)% EXEC2']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
C:\Users\carlos.ramos\AppData\Local\Temp\ipykernel_24936\2130103296.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_in.applymap(lambda v: str(v) if pd.notna(v) else v)
C:\Users\carlos.ramos\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['SETOR' 'UF' 'Atributo.1' 'Atributo.2' 'Atributo.3' 'Valor' 'SETOR2'
 'EMPREENDIMENTO2' 'Descrição2' 'Descrição']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Predições futuras salvas em: artifacts\predicoes_futuras_2026-2030.xlsx
Linhas geradas: 160
